In [ ]:
#헤드폰 연결 및 IMU, UWB 데이터 수치화
import time
import serial
import serial.tools.list_ports

for port in serial.tools.list_ports.comports():
    print(port.device)

ser = serial.Serial("/dev/cu.usbserial-110", 57600)
print(ser.name)

In [ ]:
def read_until_full(ser=ser, timeout=None):
    ser.timeout = timeout
    ser.flush()
    ser.flushInput()
    while True:
        readed = ser.read(size=1)
        if readed==b'\xff':
            incomming = ser.read(size=6)
            break 
            
    return incomming

hex_incomming = read_until_full()

[x for x in hex_incomming]

def get_values(hex_incomming):
    x = hex_incomming[0] / 10
    y = hex_incomming[1] / 10 
    heading = (hex_incomming[2]*255 + hex_incomming[3]) / 100
    return x, y, heading

get_values(hex_incomming)

def get_value_onestep():
    hex_incomming = [x for x in read_until_full()]
    #print('hex_incomming :', hex_incomming)
    return get_values(hex_incomming)

get_value_onestep()


In [ ]:
#라이브러리 선언
from flask import Flask, render_template, request, jsonify
import os
import openai
from google.cloud import speech
from google.cloud import texttospeech
import numpy as np
import sounddevice as sd
import pyaudio
import wave
import pygame
import math
import time

In [ ]:
#GPT-4 사용을 위한 인증키 가져오기
openai.api_key = "Your API Key"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Your JSON Path"

#Flask app 생성
app = Flask(__name__)
count = 0

In [ ]:
#####STT, TTS 음성 대화를 위한 코드

## 사용자 음성 녹음
def record_audio():
    CHUNK = 4096
    FORMAT = pyaudio.paInt16  # 16 bit
    CHANNELS = 1  # mono
    RATE = 16000  # sample rate
    SILENCE_THRESHOLD = 300  # silence threshold
    SILENT_CHUNKS = int(3.0 * RATE / CHUNK)  # 2.5 seconds of silence
    MAX_RECORD_SECONDS = 5  # maximum recording time in seconds

    # PyAudio 인스턴스 초기화
    p = pyaudio.PyAudio()

    # 스트림 열기
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")

    frames = []
    silent_chunks = 0
    audio_started = False
    start_time = None

    try:
        while True:
            data = stream.read(CHUNK, exception_on_overflow=False)
            frames.append(data)

            # Check for silence
            audio_data = np.frombuffer(data, dtype=np.int16)
            if np.abs(audio_data).mean() < SILENCE_THRESHOLD:
                silent_chunks += 1
                if silent_chunks > SILENT_CHUNKS:
                    break
            else:
                silent_chunks = 0   

            if audio_started:
                if silent_chunks > SILENT_CHUNKS or (time.time() - start_time) >= MAX_RECORD_SECONDS:
                    break
            elif silent_chunks == 0:
                audio_started = True
                start_time = time.time()

    finally:
        print("* done recording")

        # 스트림 종료
        stream.stop_stream()
        stream.close()
        # PyAudio 인스턴스 종료
        p.terminate()

    # Save the recorded audio to a WAV file
    wf = wave.open("audio.wav", "wb")
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b"".join(frames))
    wf.close()

    # Return the recorded audio data
    return b"".join(frames)

## Byte 오디오 데이터를 한글 text로 변환 STT
def transcribe_audio(audio_data):
    audio = speech.RecognitionAudio(content=audio_data)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="ko-KR"
    )

    response = stt_client.recognize(config=config, audio=audio)
    
    if response.results:
        return response.results[0].alternatives[0].transcript
    else:
        print("음성 인식에 실패했습니다. 다시 시도해주세요.")
        return None
    
## 텍스트를 음성으로 변환
def synthesize_speech(text, voice_index=0):
    # Ensure the provided index is valid
    if voice_index < 0 or voice_index >= len(speech_params_list):
        raise ValueError("Invalid voice index provided")

    # Choose the voice settings based on the provided index
    speech_params = speech_params_list[voice_index]

    # Convert the ssml_gender string to the corresponding enum value
    ssml_gender_map = {
        "MALE": texttospeech.SsmlVoiceGender.MALE,
        "FEMALE": texttospeech.SsmlVoiceGender.FEMALE,
    }
    ssml_gender = ssml_gender_map[speech_params["ssml_gender"]]

    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="ko-KR",
        ssml_gender=ssml_gender,  # Use the enum value instead of string
        name=speech_params["voice_name"]
    )
    
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.LINEAR16,
        speaking_rate=speech_params["speaking_rate"],
        pitch=speech_params["pitch"]
    )

    response = tts_client.synthesize_speech(
        input=input_text, voice=voice, audio_config=audio_config
    )

    return response.audio_content

## pyaudio를 사용하여 음성을 재생하는 함수
def play_audio(audio_data):
    p = pyaudio.PyAudio()
    stream = p.open(
        format=p.get_format_from_width(2),
        channels=1,
        rate=24000,
        output=True,
    )
    stream.write(audio_data)
    stream.stop_stream()
    stream.close()
    p.terminate()

# Set up STT and TTS API clients 변환된 데이터값 변수로 받기
stt_client = speech.SpeechClient()
tts_client = texttospeech.TextToSpeechClient()

# 프롬프트설계, 과거기억 데이터 확인
messages = []

In [ ]:
speech_params_list = [
    {   #0 
        "speaking_rate": 1.1,
        "pitch": -2.0,
        "ssml_gender": "MALE",
        "voice_name": "ko-KR-Standard-C"
    },
    {   #1 
        "speaking_rate": 1.0,
        "pitch": 0.0,
        "ssml_gender": "FEMALE",
        "voice_name": "ko-KR-Standard-A"
    } 
]

#while문 사용을 위한 헤딩값 재계산
def get_heading():
    return get_value_onestep()[2]   
# Zone 정보 및 좌표 설정
zones_f1 = {
    #실측 좌표값, 확장영역으로 좌표화
    "Zone1": [(4.0, 0.0), (6.0, 1.0)],  # 시작
    "Zone3": [(0.0, 0.0), (3.5, 5.5)],  # 오른쪽
    "Zone4": [(4.0, 1.0), (6.0, 5.5)],  # 중앙
    "Zone5": [(6.5, 0.0), (9.9, 5.5)],  # 왼쪽
} 
# 사운드 파일 재생 여부를 추적하기 위한 딕셔너리
sound_played = {
    "Test": False,
    "news1_rumor": False,
    "news3_rumor": False,
    "news1_murder": False,
    "news3_murder": False,
    "news2_murder": False,
    "news2_rumor": False,
    #
    "b_rec_3_0": False,
    "b_rec_3_2": False,
    "b_rec_3_3": False,
    "b_rec_3_4": False,
    #
    "b_rec_4_1": False,
    "b_rec_4_2": False
    
}
conv_executed = {
    "conv4_1": False,
    "conv4_2": False,
    "conv4_3": False,
    "conv4": False
}
zone_executed = {
    "Zone1": False,
    "Zone3": False,
    "Zone4": False,
    "Zone5": False,
    "Zone2": False,
    "Zone6": False
}

def play_bgm(path, volume=0.5):
    pygame.mixer.init()
    pygame.mixer.music.load(path)
    pygame.mixer.music.set_volume(volume)  # 볼륨 설정
    pygame.mixer.music.play(0)  # -1은 무한 반복, 0은 한번 재생을 의미

def play_conv(path, callback=None):
    pygame.mixer.init()
    pygame.mixer.music.load(path)
    pygame.mixer.music.play(0)  # 0은 파일을 한 번만 재생합니다.

    while pygame.mixer.music.get_busy():
        time.sleep(0.1)  # 음악 재생 상태 확인을 위한 잠시 대기

    if callback:  # 재생이 완료된 후 실행할 함수가 있다면 호출
        callback()

def stop_bgm():
    pygame.mixer.music.stop()

def get_zone(x, y):
    for zone, (start, end) in zones_f1.items():
        if start[0] <= x <= end[0] and start[1] <= y <= end[1]:  # 조건 수정
            return zone
    return None

zone_N = get_zone(get_value_onestep()[0], get_value_onestep()[1])
print(get_value_onestep()[0], get_value_onestep()[1])
print(zone_N)

In [ ]:
### 2.bgm 정의 및 경로수정
intro1 = "File Path"
intro2 = "File Path"

#1번 뉴스
news1 = "File Path"
news1_question = "File Path"
news1_murder = "File Path"
news1_rumor = "File Path"
#2번 뉴스
news2 = "File Path"
news2_question = "File Path"
news2_murder = "File Path"
news2_rumor = "File Path"
#3번 뉴스
news3 = "File Path"
news3_question = "File Path"
news3_murder = "File Path"
news3_rumor = "File Path"

#바뀐 생각 
news4_1_question = "File Path"
#엔딩전 질문
news4_2_question = "File Path"

#방향유도 소리 & fallback
affordance_cue = "File Path"
fallback = "File Path"

In [ ]:
import requests
import io
from pydub import AudioSegment
from pydub.playback import play

#elevenlabs
def synthesize(text):
    url = "https://api.elevenlabs.io/v1/text-to-speech/"
    headers = {
        "Accept": "audio/mpeg",
        "Content-Type": "application/json",
        "xi-api-key": "You API Key"  # 여기에 실제 API 키를 입력해야 합니다. 내 키
    }
    data = {
        "text": text,
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.75,
            "style_exaggeration": 0.0
        }
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        audio_stream = io.BytesIO(response.content)
        audio = AudioSegment.from_file(audio_stream, format="mp3")
        play(audio)
    else:
        print("Error: ", response.status_code)
        
#GPT 사전 프롬프트
assistant_role = "닥터로만 실험실 가이드, 아이"
assistant_persona = "무섭고 궁금증을 자아내는"
user_role = "방문객"
messages.append({"role": "system", "content": f"GPT(:{assistant_role})와 사용자(:{user_role})의 역할극을 시작합니다. {assistant_role}은 {assistant_persona} 성격으로 '닥터로만 실험실' 관람에 대해 스몰토크를 합니다."})
assistant_content = None

#뉴스1
def conv4_1():  
    play_conv(news1)
    news_c = "1층에 있는 로만 박사의 기괴한 작품들을 보았을때, 당신은 로만 박사가 제자의 실종과 관련이 있다고 생각하시나요?"
    synthesize(news_c)
    #play_conv(news1_question)

    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    print(user_content)

    if not user_content:
        user_content = "동의"
    messages.append({"role": "system", "content": "다음 아래 사용자 발화가 '동의'한다고 생각하는 내용이면 0.0, '동의하지 않음'이라고 생각하는 내용이면 1.0에 가깝게 수치화해서 '0.1' 이렇게 알려줘"})
    messages.append({"role": "user", "content": user_content}) #사용자 발화 추가

    # 대답1 Use GPT API to generate response
    completion = openai.ChatCompletion.create(model="gpt-4", messages=messages)
    assistant_content = completion.choices[0].message["content"].strip()
    messages.append({"role": "assistant", "content": f"{assistant_content}"})
    print(f"S : {assistant_content}")

    news_c = "발걸음 소리가 들리는 방향으로 천천히 이동해주세요."
    synthesize(news_c)

    #sentiment 변수를 선언해서 긍정부정을 0.0 ~ 1.0으로 수치화
    sentiment = assistant_content
    try:
        sentiment = float(sentiment)
    except ValueError:
        sentiment = 0.5
    return sentiment

#뉴스2
def conv4_2():  
    play_conv(news2)
    news_c2 = "세간에서는 실험을 위해 로만 박사가 일부러 제자들을 죽인다는 소문도 들리는 것 같습니다. 이 사고도 로만박사의 소행일까요? 어떻게 생각하시나요?"
    synthesize(news_c2)

    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    print(user_content)

    if not user_content:
        user_content = "동의"
    messages.append({"role": "system", "content": "다음 아래 사용자 발화가 '동의'한다고 생각하는 내용이면 0.0, '동의하지 않음'이라고 생각하는 내용이면 1.0에 가깝게 수치화해서 '0.1' 이렇게 알려줘"})
    messages.append({"role": "user", "content": user_content}) #사용자 발화 추가
    
    # 대답1 Use GPT API to generate response
    completion = openai.ChatCompletion.create(model="gpt-4", messages=messages)
    assistant_content = completion.choices[0].message["content"].strip()
    messages.append({"role": "assistant", "content": f"{assistant_content}"})
    print(f"S : {assistant_content}")

    news_c = "발걸음 소리가 들리는 방향으로 천천히 이동해주세요."
    synthesize(news_c)

    #sentiment 변수를 선언해서 긍정부정을 0.0 ~ 1.0으로 수치화
    sentiment = assistant_content
    try:
        sentiment = float(sentiment)
    except ValueError:
        sentiment = 0.5
    return sentiment

#뉴스3
def conv4_3():  
    play_conv(news3)
    news_c3 = "음.. 저 문서들의 내용이 아직 확인되지 않았지만 로만 박사에 대한 폭로 내용일까요? 어떻게 생각하시나요?"
    synthesize(news_c3)

    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    print(user_content)

    if not user_content:
        user_content = "동의"
    messages.append({"role": "system", "content": "다음 아래 사용자 발화가 '동의'한다고 생각하는 내용이면 0.0, '동의하지 않음'이라고 생각하는 내용이면 1.0에 가깝게 수치화해서 '0.1' 이렇게 알려줘"})
    messages.append({"role": "user", "content": user_content}) #사용자 발화 추가

    # 대답1 Use GPT API to generate response
    completion = openai.ChatCompletion.create(model="gpt-4", messages=messages)
    assistant_content = completion.choices[0].message["content"].strip()
    messages.append({"role": "assistant", "content": f"{assistant_content}"})
    print(f"S : {assistant_content}")

    news_c = "발걸음 소리가 들리는 방향으로 천천히 이동해주세요."
    synthesize(news_c)

    #sentiment 변수를 선언해서 긍정부정을 0.0 ~ 1.0으로 수치화
    sentiment = assistant_content
    try:
        sentiment = float(sentiment)
    except ValueError:
        sentiment = 0.5
    return sentiment

# 존1번 마지막 대화
def conv4():  
    play_conv(news4_1_question)
    news_e = "당신은 어떻게 생각하시나요?"
    synthesize(news_e)
    
    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    print(user_content)
    play_conv(news4_2_question)
    return None

In [ ]:
play_conv(affordance_cue)
play_conv(affordance_cue)

###테스트코드
current_bgm = None  # 현재 재생 중인 배경 음악을 추적하기 위한 변수
current_zone = None  # 이전 zone을 추적하기 위한 변수
previous_zone = None
condition = None # 경로가 5-6-5, 3-2-3 이면 condition 값을 1로 변경
sentiment = None # 살인자면 0, 루머면 1
count_z = 0
tracking_zone = None #존 추적

In [ ]:
get_value_onestep()

In [ ]:
#경험 시작 인트로
play_conv(intro1)
play_conv(intro2)
#synthesize("소리를 찾아 이동해주세요.")

step = 1

values_list = []

while True:
    array_t = get_value_onestep()
    pre_x = array_t[0]
    pre_y = array_t[1]
    heading = array_t[2]
    values_list.append(array_t)

    current_zone = get_zone(pre_x, pre_y)

    print("헤딩:", heading) #연구실 환경에 맞게 0도를 화이트보드 방향으로 
    print("좌표:", pre_x, ",", pre_y)
    print(f"현재위치: {current_zone}.")

    # 현재 Zone 결정 및 이전 Zone 추적
    if current_zone != previous_zone:
        previous_zone = current_zone  # 현재 Zone을 이전 Zone으로 업데이트

    if current_zone == "Zone1":
        """if step == 4:
            print("1")
            conv4()
            conv_executed["conv4"] = True
        elif 350 < heading < 360 or 0 < heading < 20:
            print("2")"""
        if 300 < heading < 360 or 0 < heading < 40:
            print("1")
            play_conv(affordance_cue)
            play_conv(affordance_cue)

    elif current_zone == "Zone4":
        print("3")
        print(step)
        print(f"현재 tracking_zone: {tracking_zone}, count_z: {count_z}")
        print("작동중이면, 값이 계속 변함: ", pre_x, pre_y)
        if step == 1:
            if not conv_executed["conv4_1"]:
                print("4")
                sentiment = conv4_1()
                sentiment = float(sentiment)
                conv_executed["conv4_1"] = True  # conv4_1 실행 표시
            elif tracking_zone in ["Zone3", "Zone5"] and count_z == 0:
                step = 2
            else:
                print("4-1")
                if not isinstance(sentiment, float):
                    sentiment = 0.5
                if 0.0 <= sentiment <= 0.5 and 240 < heading < 300: #존5 - 살인
                    play_conv(affordance_cue)
                elif 0.5 < sentiment <= 1.0 and 60 < heading < 120: #존3 - 소문
                    play_conv(affordance_cue)
        elif step == 2:
            if not conv_executed["conv4_2"]:
                print("6")
                sentiment = conv4_2()
                sentiment = float(sentiment)
                conv_executed["conv4_2"] = True 
            elif tracking_zone in ["Zone3", "Zone5"] and count_z == 1:
                step = 3
            else:
                print("4-2")
                if not isinstance(sentiment, float):
                    sentiment = 0.5
                if 0.0 <= sentiment <= 0.5 and 240 < heading < 300: #존5 - 살인
                    play_conv(affordance_cue)
                elif 0.5 < sentiment <= 1.0 and 60 < heading < 120: #존3 - 소문
                    play_conv(affordance_cue)
        elif step == 3:
            if not conv_executed["conv4_3"]:
                print("7")
                sentiment = conv4_3()
                sentiment = float(sentiment)
                conv_executed["conv4_3"] = True 
            elif tracking_zone in ["Zone3", "Zone5"] and count_z == 2:
                step = 4
            else:
                print("4-3")
                if not isinstance(sentiment, float):
                    sentiment = 0.5
                if 0.0 <= sentiment <= 0.5 and 240 < heading < 300: #존5 - 살인
                    play_conv(affordance_cue)
                elif 0.5 < sentiment <= 1.0 and 60 < heading < 120: #존3 - 소문
                    play_conv(affordance_cue)
        elif step == 4:
            print("8")
            conv4()
            conv_executed["conv4"] = True
            break
            """if 160 < heading < 200: #존 1로 보내기
                play_conv(affordance_cue)"""

    elif current_zone == "Zone3":
        print("작동중이면, 값이 계속 변함: ", pre_x, pre_y)
        if step == 1 and not sound_played["news1_rumor"]:
            news1_rumor ="로봇 기계는 인간의 신체처럼 정교하고, 정성을 다해 만들어야 한다는 것이, 로만 박사님의 신조였습니다. 그는 인간의 근육운동을 재현하기 위해 수십개의 근육을 실제로 구현한 집요한 연구자 였어요."
            synthesize(news1_rumor)
            news1_foot ="다음 발걸음 소리가 들리는 방향으로 이동해 주세요."
            synthesize(news1_foot)
            sound_played["news1_rumor"] = True
            tracking_zone = "Zone3"
            if 240 < heading < 300: #존 4로 보내기
                play_conv(affordance_cue)
        elif step == 2 and not sound_played["news2_rumor"]:
            news2_rumor ="박사가 사람 모양의 조각을 만든 이유는, 인간에 가까운 정교한 로봇을 만들기 위함이 아니었을까요?, 단지 영감을 얻기 위해서는 아니었을까요?, 아직 다른 소리가 남았습니다. 발걸음 소리가 들리는 방향으로 천천히 이동해 주세요."
            synthesize(news2_rumor)
            sound_played["news2_rumor"] = True
            tracking_zone = "Zone3"
            count_z = 1
            if 240 < heading < 300: #존 4로 보내기
                play_conv(affordance_cue)
        elif step == 3 and not sound_played["news3_rumor"]:
            news3_rumor ="박사는 로봇공학의 권위자로, 조금 기괴하고 특이한 신체 조각을 만드는 취미가 있다는 소문은 많은 사람들이 알고 있었다고 하죠."
            synthesize(news3_rumor)
            news1_foot ="다음 발걸음 소리가 들리는 방향으로 이동해 주세요."
            synthesize(news1_foot)
            sound_played["news3_rumor"] = True
            tracking_zone = "Zone3"
            count_z = 2
            if 240 < heading < 300: #존 4로 보내기
                play_conv(affordance_cue)
        elif 240 < heading < 300: #존 4로 보내기
            tracking_zone = "Zone3"
            play_conv(affordance_cue)

    elif current_zone == "Zone5":
        print("작동중이면, 값이 계속 변함: ", pre_x, pre_y)
        if step == 1 and not sound_played["news1_murder"]:
            news1_rumor ="그는 인간과 똑같은 로봇에 만드는 것에 집착했다고 합니다. 1층에서 본 조각처럼요. 그가 인체실험을 위해 사람은 실제로 죽인 것은 아니었을까요?"
            synthesize(news1_rumor)
            news1_foot ="다음 발걸음 소리가 들리는 방향으로 이동해 주세요."
            synthesize(news1_foot)
            sound_played["news1_murder"] = True
            tracking_zone = "Zone5"
            if 60 < heading < 120: #존 4로 보내기
                play_conv(affordance_cue)
        elif step == 2 and not sound_played["news2_murder"]:
            news2_rumor ="1층 조각들을 잘 생각해보세요. 팔다리가 따로 있는 작품들과, 팔다리가 없는 작품들이 있었죠. 혹시 누군가의 신체 일부가 아니었을까요?, 아직 다른 소리가 남았습니다. 발걸음 소리가 들리는 방향으로 이동해 주세요."
            synthesize(news2_rumor)
            sound_played["news2_murder"] = True
            tracking_zone = "Zone5"
            count_z = 1
            if 60 < heading < 120: #존 4로 보내기
                play_conv(affordance_cue)
        elif step == 3 and not sound_played["news3_murder"]:
            news3_rumor ="사실 로만 박사의 조각은 진짜 사람 신체로 만들었다는 소문은 예전부터 들려왔습니다. 죽은 제자의 문서에는 박사의 만행을 폭로하는 내용이 담겨 있었을까요?."
            synthesize(news3_rumor)
            news1_foot ="다음 발걸음 소리가 들리는 방향으로 이동해 주세요."
            synthesize(news1_foot)
            sound_played["news3_murder"] = True
            tracking_zone = "Zone5"
            count_z = 2
            if 60 < heading < 120: #존 4로 보내기
                play_conv(affordance_cue)
        elif 60 < heading < 120: #존 4로 보내기
            tracking_zone = "Zone5"
            play_conv(affordance_cue)
    else:
        if 60 < heading < 120: #존 4로 보내기
            play_conv(affordance_cue)

In [ ]:
stop_bgm()   #인트로 37

In [ ]:
values_list

지하 1층 bgm.
https://music.youtube.com/watch?v=WBbVUDVJb3M